In [1]:
%load_ext autoreload
%autoreload

In [2]:
import os
import sys

from osgeo import osr, ogr, gdal
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import math
import time
import pandas as pd
from scipy import interpolate
import json
from matplotlib import colors as mcolors
import matplotlib.image as mpimg
import matplotlib
from shapely import geometry
import time 
import pyproj
from shapely.affinity import rotate
from shapely.geometry import Point, LineString, Polygon
plt.ion()
%matplotlib qt
matplotlib.use("Qt5Agg")

sys.path.append('/media/sf_VBox_Shared/PAOLA/PRISMA3/')

from PRISMA_SDS import PRIS_tools_v2, PRIS_img_v2, PRIS_profiles_v2, PRIS_Shoreline_v2

In [3]:
def GridRaster(dir_img, crs):
    
    """
    Stores the image cells into a pandas geodatabase (cells as polygon features)
    """
    
    raster = gdal.Open(dir_img)
    geoT = raster.GetGeoTransform()
    pixWidth = geoT[1]
    n_cols = raster.RasterXSize
    n_rows = raster.RasterYSize
    
    row = []
    col = []
    pol = []
    
    for r in range(n_rows):
        for c in range(n_cols):
            
            Xcoord, Ycoord = PRIS_tools_v2.pixel_to_world(raster, c, r)
            upper_left = [Xcoord, Ycoord]
            bottom_left = [Xcoord, Ycoord - pixWidth]
            upper_right = [Xcoord + pixWidth, Ycoord]
            bottom_right = [Xcoord + pixWidth, Ycoord - pixWidth]
            
            poly = geometry.Polygon([bottom_left, upper_left, upper_right, bottom_right, bottom_left])
            
            pol.append(poly)
            row.append(r)
            col.append(c)
            
    dataframe = pd.DataFrame({'row':row, 'col':col, 'geometry':pol})
    grid = gpd.GeoDataFrame(dataframe, crs = crs)
    
    return grid

In [4]:
def ProfileBounds(pr):
    
    """
    Returns the profile BB
    """
    
    xmin = pr.bounds['minx'].values[0]
    xmax = pr.bounds['maxx'].values[0]
    ymin = pr.bounds['miny'].values[0]
    ymax = pr.bounds['maxy'].values[0]
    
    return xmin, xmax, ymin, ymax

def ProfileBeginning(pr):
    
    """
    Gives the profile starting coordinate
    """
    
    pr.reset_index(inplace = True)
    coords = [(coords) for coords in list(pr.geometry[0].coords)]
    x1 = coords[0][0]
    y1 = coords[0][1]
    
    return x1, y1

def EuclideanDistance(x1, y1, x2, y2):
    
    """
    Calculates the euclidean distance between 2 points
    
    """
 
    dis_X = (x1 - x2)**2
    dis_Y = (y1 - y2)**2
    dist = np.sqrt(dis_X + dis_Y)

    return dist

def MidPointDistances(overlayed, x_prx, y_prx):
    
    """
    Obtain the midpoint of a segment
    
    """
    
    distances = []

    for point in overlayed['midpoint']:

        mid_point = list(point.coords)
        x2, y2 = mid_point[0][0], mid_point[0][1]

        distances.append(EuclideanDistance(x_prx, y_prx, x2 ,y2))
        
    overlayed['dist_midpoint'] = distances
    overlayed.sort_values(by = 'dist_midpoint', ascending = True, inplace = True)
    
    return overlayed

In [5]:
def MeanReflectances(overlayed, arr):
    
    """
    Add to the overlated geodatabase the mean reflectance value of the particular pixel
    """
    
    mean_reflectance = []

    for r, c in zip(overlayed['row'], overlayed['col']):
        ss = arr[r,c,:]
        mean_reflectance.append(np.mean(ss))

    overlayed['mean_R'] = mean_reflectance
    
    return overlayed

def Interpolate_R_PR(overlayed, interval):
    
    x_old = np.array(overlayed['dist_midpoint'])
    y_old = np.array(overlayed['mean_R'])
    
    x_new = np.arange(x_old[0], x_old[-1], interval)
    f = interpolate.interp1d(x_old, y_old, kind = 'cubic')
    y_new =  f(x_new)
    
    deriv_new = y_new[1:] - y_new[:-1]
    idd_inter_new = np.argmin(deriv_new)
    inter = y_new[idd_inter_new]
    
    return idd_inter_new, inter, x_new, y_new, deriv_new

def SSInterfacePixe(GeoT, arr, lon, lat):

    # Pixel containing the transition coordinate
    
    col, row = PRIS_tools_v2.world_to_pixel(GeoT, lon, lat)
    
    # Get the spectral signature
    
    ss = arr[row, col, :]
    
    return col, row, ss
    
    

def PlotProfileValues(paths, date, idd_inter_new, inter, x_new, y_new, deriv_new, pr):

        
    pathS0 = os.path.join(paths['Interface_pix_plot'], 'MeanSS_pr')
    
    if not os.path.exists(pathS0):
        os.mkdir(pathS0)
    
    
    name = 'MeanSS_'+ pr + '_' + date + '.png'

    pathS = os.path.join(pathS0, date)

    if not os.path.exists(pathS):
        os.mkdir(pathS)


        
    plt.scatter(idd_inter_new, inter, s=150, c='firebrick', alpha = 0.5, label = 'interface')
    plt.plot(y_new, marker = 'o', color='royalblue', label = 'mean reflectance') # royalblue
    plt.plot(deriv_new, marker = 'o', color = 'slategray', label = 'derived') # #  skyblue

    ax.spines['top'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_color('grey')
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('grey')
    ax.grid(color='gainsboro', linestyle = '--', axis = 'y')

    plt.title('Profile '+ pr, fontsize = 15)
    plt.xlabel("d (m)", fontsize = 15)
    plt.ylabel("Mean Reflectance", fontsize = 15)
    plt.legend()
    plt.savefig(os.path.join(pathS, name))
    plt.show()
    plt.close()
    

In [6]:
def AzimuthAngle(coor_pr_i, coor_pr_f):
    angle = (180/np.pi) * math.atan2(coor_pr_f[0] - coor_pr_i[0], coor_pr_f[1] -coor_pr_i[1])
    
    if angle < 0:
        angle = (angle + 360)%360
    return(angle)

In [7]:
def dot(vA, vB):
    return vA[0]*vB[0] + vA[1] * vB[1]

def Cal_ang(lineA, lineB):
    vA = [(lineA[0][0] - lineA[1][0]), (lineA[0][1] - lineA[1][1])]
    vB = [(lineB[0][0] - lineB[1][0]), (lineB[0][1] - lineB[1][1])]

    # Get dot profuct
    dot_pro = dot(vA, vB)
    # Get magnitudes
    magA = dot(vA, vA)**0.5
    magB = dot(vB, vB)**0.5
    # Get cosine value
    cos_ = dot_pro/magA/magB
    
    # Get angle in radians and then convert to degrees
    if dot_pro == 0:

        ang_deg = np.degrees(np.arccos(0)) 
    else:
        angle = math.acos(dot_pro/magB/magA)
        # Doing angle <- angle mod 360
        ang_deg = math.degrees(angle)%360

    if ang_deg - 180 >= 0:
        return 360 - ang_deg
    else:
        return ang_deg

In [8]:
# This function is used only to check if a profiles is fully contained on the image BB

def ImageBB(paths, date):
    
    imgName = [f for f in os.listdir(paths['PanSharp_RGB']) if f.endswith('.tiff') and date in f][0]
    img = gdal.Open(os.path.join(paths['PanSharp_RGB'], imgName))

    nC = img.RasterXSize
    nR = img.RasterYSize

    geoT = img.GetGeoTransform()
    res = geoT[1]
    xmin = min(geoT[0], geoT[0] + nC * geoT[1])
    xmax = max(geoT[0], geoT[0] + nC * geoT[1])
    ymin = min(geoT[3], geoT[3] + nR * geoT[5])
    ymax = max(geoT[3], geoT[3] + nR * geoT[5])
    north = [(xmin, ymin), (xmin, ymax)]
    
    return xmin, xmax, ymin, ymax, north

In [9]:
def AnaliziyngProfiles(profiles, date, grid, arr, img_BB, GeoT, plot):
    
    # Generate the DataFrame

    lon = []
    lat = []
    
    cols = []
    rows = []
    SS = []
    PRS = []

    for pr in profiles['PR']:
        

        print("", end=f"\r Analyzing {pr}")
        
        # Search the profile

        prx = profiles[profiles['PR'] == pr]
        prx.reset_index(inplace = True, drop = True)
        
        if not img_BB.contains(prx['geometry'][0]):
            continue

        # Get pixel initial coordinate

        x_prx, y_prx = ProfileBeginning(prx)

        # Overlay the grid and the profiles geometries

        overlayed = gpd.overlay(prx, grid, how = 'intersection')
        overlayed.reset_index(inplace = True, drop = True)

        ## Check that the length of the obtained pixels is the same of the entire profile
        

        assert [np.abs(overlayed['geometry'].length.sum() - prx['geometry'].length) < 1.e-6][0][0]

        ## Check that all the distances are smaller than 5*sqrt(2)

        overlayed['length'] =  [line.length for line in overlayed['geometry']]
        overlayed['bool'] = overlayed['length'] < 5 * np.sqrt(2)
        

        assert overlayed['bool'].unique()[0]

        ## Obtain the mid point of the segments

        overlayed['midpoint'] = overlayed.apply(lambda row: row['geometry'].centroid, axis=1) #Find centroid

        ## Calculate the distances of each midpoint to the beginning profile point

        overlayed = MidPointDistances(overlayed, x_prx, y_prx)

        ## Add the mean reflectances
        

        overlayed = MeanReflectances(overlayed, arr)

        ## Interpolate distances an reflectances

        idd_inter_new, inter, x_new, y_new, deriv_new = Interpolate_R_PR(overlayed, 0.25)
        
        Save_plotInfo(paths, date, pr, idd_inter_new, inter, x_new, y_new, deriv_new )

        ## Save and generate the plots
        
        if plot:

            PlotProfileValues(paths, date, idd_inter_new, inter, x_new, y_new, deriv_new, pr) 

        # Now we know which is the point we can obtain the shoreline

        new_length = x_new[idd_inter_new]

        # We have to know which is the profile angle to be able to 
        
        xmin_pr, xmax_pr, ymin_pr, ymax_pr = ProfileBounds(prx)
        
        north = [(xmin_pr, ymin_pr), (xmin_pr, ymax_pr)] # nortth direction
        
        coords = [(coords) for coords in list(prx.geometry[0].coords)]
        coor_pr_i, coor_pr_f = [coords[i] for i in (0,-1)]
        slp = (coor_pr_f[1] - coor_pr_i[1]) / (coor_pr_f[0] - coor_pr_i[0]) # slope profile
        pr_line = [(coor_pr_i[0], coor_pr_i[1]), (coor_pr_f[0], coor_pr_f[1])]
        
        angle = AzimuthAngle(coor_pr_i, coor_pr_f)
        Deg_rot = Cal_ang(north, pr_line)
        
        start = Point(coor_pr_i[0], coor_pr_i[1])
        end = Point(start.x  , start.y + new_length)
        Line = LineString([start, end])
        

        if (90 < angle <270) and (slp < 0):

            pr_new = rotate(Line , -Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

        if (180 < angle <270) and (slp > 0):

            pr_new = rotate(Line , Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

        if (0 < angle <90) and (slp > 0):

            pr_new = rotate(Line , -Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)
            
        if (270< angle < 360) and (slp < 0):
            
            pr_new = rotate(Line , Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)
        
        
        coords_n = [(coords) for coords in list(pr_new.coords)]
        coor_pr_iN, coor_pr_fN = [coords_n[i] for i in (0,-1)]
        
        col, row, ss = SSInterfacePixe(GeoT, arr, coor_pr_fN[0], coor_pr_fN[1])
        
        cols.append(col)
        rows.append(row)
        SS.append(list(ss))
        PRS.append(pr)

        lon.append(coor_pr_fN[0])
        lat.append(coor_pr_fN[1])
        
    interface_pix = pd.DataFrame({'pr':PRS, 'row':rows, 'col':cols, 'signature':SS})

    return lon, lat, interface_pix


In [10]:
def Save_plotInfo(paths, date, pr, idd_inter_new, inter, x_new, y_new, deriv_new ):
    
    filename = '{:s}_RatioR_{:d}_{:.20f}.txt'.format(pr,idd_inter_new, inter)
    pathS0 = paths['Interface_pix_txtPlot']
    
    pathS = os.path.join(pathS0, date)

    if not os.path.exists(pathS):
        os.mkdir(pathS)
    
    dir_fileout = os.path.join(pathS, filename)
    
    #fileout = open(dir_fileout, "w")
    
    b = deriv_new[-1]
    
    deriv_new2 = np.append(deriv_new,b)
    
    info = pd.DataFrame({'x':x_new, 'y':y_new, 'deriv':deriv_new2})
    info.to_csv(dir_fileout, index = False)
    
    #for i in range(len(x_new)):

        #fileout.write('{:9.3f} {:9.3f} {:9.3f}\n'.format(x_new[i], y_new[i], deriv_new2[i]))
                         
    #fileout.close()
        

In [11]:
def Generate_Dict_Training_Dict(path_trainig):
    
    "Load the mean median vlues"
    
    info = pd.read_csv(os.path.join(path_trainig))
    groups = info.groupby('Proxy')
    dictio = {}
    for idd, group in groups:
        dictio[idd] = group['Medians'].quantile(q=0.9)
        
    return dictio


def turning_points(array):
    
    idx_max, idx_min = [], []
    if (len(array) < 3):
        return idx_min, idx_max
    NEUTRAL, RISING, FALLING = range(3)
    
    def get_state(a,b):
        if a < b: return RISING
        if a > b: return FALLING
        return NEUTRAL
    ps = get_state(array[0], array[1])
    begin = 1
    for i in range(2, len(array)):
        s = get_state(array[i-1], array[i])
        if s!= NEUTRAL:
            if ps != NEUTRAL and ps != s:
                if s == FALLING:
                    idx_max.append((begin + i - 1) // 2)
                else:
                    idx_min.append((begin + i - 1) // 2)
            begin = i
            ps = s
    return idx_min, idx_max

def MinimumPerSegment(indexes_turning, x_split, y_split):
    
    idds_inter = []
    inters_seg = []
    cols = []
    rows = []
    for a , b in zip(indexes_turning, indexes_turning[1:]):
        
        if b - a == 1:
            continue

        x_seg = x_split[a:b]
        y_seg = y_split[a:b]
        idd_inter_seg, inter_seg, x_new_seg, y_new_seg, deriv_new_seg = Interpolate_R_PR_Split(x_seg, y_seg)
        idds_inter.append(a + idd_inter_seg)
        inters_seg.append(inter_seg)
        
    return idds_inter, inters_seg

def Interpolate_R_PR_Split(x_split, y_split):
    
    x_new = x_split
    y_new = y_split
    
    #x_new = np.arange(x_old[0], x_old[-1])
    #f = interpolate.interp1d(x_old, y_old, kind = 'cubic')
    #y_new =  f(x_new)
    
    deriv_new = y_new[1:] - y_new[:-1]
    idd_inter_new = np.argmin(deriv_new)
    inter = y_new[idd_inter_new]
    
    return idd_inter_new, inter, x_new, y_new, deriv_new 

In [12]:
path0 = '/media/sf_VBox_Shared/PAOLA/PRISMA_FOAM/'
scene = 'GR'

beaches = [f for f in os.listdir(os.path.join(path0, 'SCENES', scene)) if 'GR01' in f] #if '01' in f

In [13]:
u = 0

for beach in beaches:
    
    print(beach)
    
    paths = PRIS_tools_v2.generate_structure(path0, scene, beach)
    
    dir_inter_Rat_txtPlot = os.path.join(paths['scene'], beach, 'Results', 'Interface_pix', 'txtPlot')
    
    if not os.path.exists(dir_inter_Rat_txtPlot):
        os.makedirs(dir_inter_Rat_txtPlot)
        
    
    paths['Interface_pix_txtPlot'] = dir_inter_Rat_txtPlot
    
    imgs = [f for f in os.listdir(paths['PanSharp_Square_Cropped']) if f.endswith('tiff')]
    
    
    for imagename in imgs:
        
        date = imagename.split('_')[1]
        
        if date == '20201015':
            
            continue
        
        print('Analyzing imgage: ', str(imagename), flush = True)
      
        
        dir_img =  os.path.join(paths['PanSharp_Square_Cropped'], imagename)
        print('leo imagen')
        arr, arr_rgb = PRIS_img_v2.numpy_from_tiff(dir_img, True)
        print('he leido imagen')
        img = gdal.Open(dir_img)
        GeoT = img.GetGeoTransform()
        proj = osr.SpatialReference(wkt=img.GetProjection())
        crs = pyproj.CRS("EPSG:" + proj.GetAttrValue('AUTHORITY',1))

        
        # CREATE THE RASTER GRID GEODATAFRAME
        
        #grid = GridRaster(dir_img, crs)

        dir_grids = os.path.join(paths['scene'] , beach, 'QGIS', 'Img_grid')
        gridName = [f for f in os.listdir(dir_grids) if date in f][0]
        grid = gpd.read_file(os.path.join(dir_grids, gridName))
        
        # Image Bounding Box
        
        xmin, xmax, ymin, ymax, north = ImageBB(paths, date)
        img_BB = Polygon([[xmin,ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])
        
        ## LOAD THE PROFILES
        
        shapefiles = [f for f in os.listdir(paths['Profiles']) if f.endswith('.shp') and date in f]

GR01
Analyzing imgage:  SPI_20201115_PRISMA_PANS_0050_0918_00084_00078.tiff
leo imagen
he leido imagen


In [14]:
u = 0

# if there is more than one profile per date (inclination study) we must iterate throught

for shapefile in shapefiles:

    print('SDS for profiles: ', shapefile)

    filename = imagename.split('.')[0] + '_' + 'profiles_'+   '_'.join(shapefile.split('_')[2:4]) + '.txt'

    #if os.path.exists(os.path.join(paths['SDS'], 'derivative', filename)):
        #print('Already calculated')
        #continue

    profile_path = os.path.join(paths['Profiles'],shapefile)


    if os.path.exists(profile_path):
        print('Load the profiles')
        profiles = gpd.read_file(profile_path, encoding="utf-8")
        crs_pr = profiles.crs
        
    
        
    for pr in profiles['PR']:

        if u == 3:  

            print("", end=f"\r Analyzing {pr}")

            # Search the profile

            prx = profiles[profiles['PR'] == pr]
            prx.reset_index(inplace = True, drop = True)

            if not img_BB.contains(prx['geometry'][0]):
                continue

            # Get pixel initial coordinate

            x_prx, y_prx = ProfileBeginning(prx)

                    # Genearte grid subset

            x1, y1 = prx.boundary[0][0].xy[0][0], prx.boundary[0][0].xy[1][0]
            x2, y2 = prx.boundary[0][1].xy[0][0], prx.boundary[0][1].xy[1][0]
            col1, row1 = PRIS_tools_v2.world_to_pixel(GeoT, x1, y1)
            col2, row2 = PRIS_tools_v2.world_to_pixel(GeoT, x2, y2)

            rowmin = min(row1, row2)
            rowmax = max(row1, row2)
            colmin = min(col1, col2)
            colmax = max(col1, col2)

            new_grid = grid[(grid['row'] >= rowmin) & (grid['row']<=rowmax) & (grid['col'] <= colmax) & (grid['col'] >= colmin)]
            new_grid.reset_index(inplace = True, drop = True)


            # Overlay the grid and the profiles geometries

            overlayed = gpd.overlay(prx, new_grid, how = 'intersection')
            overlayed.reset_index(inplace = True, drop = True)

            ## Check that the length of the obtained pixels is the same of the entire profile


            assert [np.abs(overlayed['geometry'].length.sum() - prx['geometry'].length) < 1.e-6][0][0]

            ## Check that all the distances are smaller than 5*sqrt(2)

            overlayed['length'] =  [line.length for line in overlayed['geometry']]
            overlayed['bool'] = overlayed['length'] < 5 * np.sqrt(2)


            assert overlayed['bool'].unique()[0]

            ## Obtain the mid point of the segments

            overlayed['midpoint'] = overlayed.apply(lambda row: row['geometry'].centroid, axis=1) #Find centroid

            ## Calculate the distances of each midpoint to the beginning profile point

            overlayed = MidPointDistances(overlayed, x_prx, y_prx)

            ## Add the mean reflectances


            overlayed = MeanReflectances(overlayed, arr)
            overlayed['distances'] = overlayed['length'].cumsum()


            ## Interpolate distances an reflectances

            idd_inter_new, inter, x_new, y_new, deriv_new = Interpolate_R_PR(overlayed, 0.25)

           
                ## Generate Dataframe

            b = deriv_new[-1]
            deriv_new2 = np.append(deriv_new,b)
            Interpolat = pd.DataFrame({'dis': x_new, 'mean_r':y_new, 'deriv':deriv_new2})

            # Define which points are foam and which not

            thre = 0.12

            Interpolat['Bool_Foam'] = Interpolat['mean_r'] <= thre
            Interpolat.sort_values(by = 'dis', ascending = True, inplace = True)
            
        u = u + 1


SDS for profiles:  SPI_20201115_-90_0_pr.shp
Load the profiles
 Analyzing PR4

In [16]:
cut_index = Interpolat[Interpolat['Bool_Foam']].index.min()

In [17]:
if np.isnan(cut_index):
    Inter = Interpolat
else:
    # Interpolated Sand

    Inter = Interpolat[0:cut_index]

In [ ]:


## Get all the values before, which are supposed to be also sand

assert np.unique(Inter['mean_r'] > thre)

deriv_new = Inter['deriv']
idx_min, idx_max = turning_points(deriv_new)

if len(idx_min) == 0:
    print('Es 0')

## Ahora tengo que ver cual es el minimo de los turning points minimos

Turning_values = [Inter.iloc[int(f)]['deriv'] for f in idx_min]

ff = pd.DataFrame({'values':Turning_values, 'idx':idx_min})

## Which is the inter value and where is located (needed to save the info)

idd_inter_new = ff.sort_values(by=['values','idx'], ascending = True)['idx'].iloc[0]

x_new = Inter['dis']

new_length = x_new[idd_inter_new]


In [34]:
paths['SDS']

'/media/sf_VBox_Shared/PAOLA/PRISMA_FOAM/SCENES/GR/GR01/Results/SDS'

In [26]:
xmin_pr, xmax_pr, ymin_pr, ymax_pr = ProfileBounds(prx)
north = [(xmin_pr, ymin_pr), (xmin_pr, ymax_pr)] # nortth direction

coords = [(coords) for coords in list(prx.geometry[0].coords)]
coor_pr_i, coor_pr_f = [coords[i] for i in (0,-1)]
slp = (coor_pr_f[1] - coor_pr_i[1]) / (coor_pr_f[0] - coor_pr_i[0]) # slope profile
pr_line = [(coor_pr_i[0], coor_pr_i[1]), (coor_pr_f[0], coor_pr_f[1])]

angle = AzimuthAngle(coor_pr_i, coor_pr_f)
Deg_rot = Cal_ang(north, pr_line)

start = Point(coor_pr_i[0], coor_pr_i[1])
end = Point(start.x  , start.y + new_length)
Line = LineString([start, end])

if (90 < angle <270) and (slp < 0):

    pr_new = rotate(Line , -Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

if (180 < angle <270) and (slp > 0):

    pr_new = rotate(Line , Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

if (0 < angle <90) and (slp > 0):

    pr_new = rotate(Line , -Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

if (270< angle < 360) and (slp < 0):

    pr_new = rotate(Line , Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)
    
coords_n = [(coords) for coords in list(pr_new.coords)]
coor_pr_iN, coor_pr_fN = [coords_n[i] for i in (0,-1)]

col, row, ss = SSInterfacePixe(GeoT, arr, coor_pr_fN[0], coor_pr_fN[1])

lon = coor_pr_fN[0]
lat = coor_pr_fN[1]


NameError: name 'new_length' is not defined

In [88]:
lon, lat

(442519.3726496943, 6156823.758376219)

In [14]:
def GetPixCoordinates(idd_inter_new, inter, x_new, y_new, deriv_new, prx):
    
    new_length = x_new[idd_inter_new]

    # We have to know which is the profile angle to be able to 

    xmin_pr, xmax_pr, ymin_pr, ymax_pr = ProfileBounds(prx)

    north = [(xmin_pr, ymin_pr), (xmin_pr, ymax_pr)] # nortth direction

    coords = [(coords) for coords in list(prx.geometry[0].coords)]
    coor_pr_i, coor_pr_f = [coords[i] for i in (0,-1)]
    slp = (coor_pr_f[1] - coor_pr_i[1]) / (coor_pr_f[0] - coor_pr_i[0]) # slope profile
    pr_line = [(coor_pr_i[0], coor_pr_i[1]), (coor_pr_f[0], coor_pr_f[1])]

    angle = AzimuthAngle(coor_pr_i, coor_pr_f)
    Deg_rot = Cal_ang(north, pr_line)

    start = Point(coor_pr_i[0], coor_pr_i[1])
    end = Point(start.x  , start.y + new_length)
    Line = LineString([start, end])


    if (90 < angle <270) and (slp < 0):

        pr_new = rotate(Line , -Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

    if (180 < angle <270) and (slp > 0):

        pr_new = rotate(Line , Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

    if (0 < angle <90) and (slp > 0):

        pr_new = rotate(Line , -Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)

    if (270< angle < 360) and (slp < 0):

        pr_new = rotate(Line , Deg_rot , origin = [coor_pr_i[0], coor_pr_i[1]], use_radians = False)


    coords_n = [(coords) for coords in list(pr_new.coords)]
    coor_pr_iN, coor_pr_fN = [coords_n[i] for i in (0,-1)]

    col, row, ss = SSInterfacePixe(GeoT, arr, coor_pr_fN[0], coor_pr_fN[1])

    return col, row

In [15]:
def Interpolate_R_PR_Split(x_split, y_split):
    
    x_new = x_split
    y_new = y_split
    
    #x_new = np.arange(x_old[0], x_old[-1])
    #f = interpolate.interp1d(x_old, y_old, kind = 'cubic')
    #y_new =  f(x_new)
    
    deriv_new = y_new[1:] - y_new[:-1]
    idd_inter_new = np.argmin(deriv_new)
    inter = y_new[idd_inter_new]
    
    return idd_inter_new, inter, x_new, y_new, deriv_new 

In [16]:
y_split = y_new[0:idd_inter_new - 1]
x_split = x_new[0:idd_inter_new - 1]
deriv_split = deriv_new[0:idd_inter_new - 1]
idd_inter_new_split, inter_split, x_new_split, y_new_split, deriv_new_split = Interpolate_R_PR_Split(x_split, y_split)

In [17]:
inter

0.012683478510915697

In [18]:
for idd, line in overlayed.iterrows():
    if ((row == line['row']) & (col == line['col'])):
        print('si')

si


In [164]:

arr_copy = arr_rgb.copy()

for idd, line in overlayed.iterrows():
        arr_copy[line['row'], line['col'], :] = [255, 255, 255]
        
with plt.rc_context({'axes.edgecolor':'grey', 'xtick.color':'grey', 'ytick.color':'grey'}):
    
    fig, ax = plt.subplots(2,2, sharex = True, figsize = (14,6))

    ax[0,0].plot(x_new,y_new, marker = 'o',markersize = 2.5, color='orange', label = 'Averaged R') # royalblue
    ax[0,0].scatter(x_new[idd_inter_new], inter, s=50, c='firebrick', alpha = 0.8, label = 'Interface Pix',
                   zorder = 3)

    #plt.plot(deriv_new/0.25, marker = '.', color = 'slategray', label = 'derived') # #  skyblue





    #ax[0].plot(y_split, color = 'orange')
    #ax[1].scatter(x_new[idd_inter_new], inter_split, s=150, c='firebrick', alpha = 0.5, label = 'interface')
    #plt.plot((deriv_new_split/0.25)*2, marker = '.', color = 'slategray', label = 'derived') # #  skyblue
    ax[1,0].plot(x_new[:-1], deriv_new/0.25, marker = '.', color = 'steelblue', label = 'derived') # #  skyblue

    ax[0,0].axvspan(0, distances[0][0], alpha = 0.6, facecolor = colors1[0].data)
    ax[1,0].axvspan(0, distances[0][0], alpha = 0.6, facecolor = colors1[0].data)
    o = 1
    for diss, collor in zip(distances[0], colors1):
        #print(matplotlib.colors.to_hex(collor))

        if o == 42:
            o = 41
        ax[0,0].axvspan(diss, distances[0][o], alpha = 0.6, facecolor = collor.data)
        ax[1,0].axvspan(diss, distances[0][o], alpha = 0.6, facecolor = collor.data)

        ax[0,0].axvline(diss, color= 'white', ls = 'dashed', linewidth = 0.2, zorder = 3)
        ax[1,0].axvline(diss, color= 'white', ls = 'dashed', linewidth = 0.2, zorder = 3)
        o += 1

    plt.subplot(1,2,2)
    plt.imshow(arr_copy)
    plt.plot(col, row, 'o', markersize = 4, c = 'firebrick', label = 'Interface Pix')
    plt.legend(loc = 'lower right')
    plt.axis('off')

    ax[0,0].spines['top'].set_visible(False)
    ax[1,0].spines['top'].set_visible(False)
    ax[0,0].spines['bottom'].set_color('grey')
    ax[1,0].spines['bottom'].set_color('grey')
    ax[0,0].spines['right'].set_visible(False)
    ax[1,0].spines['right'].set_visible(False)
    ax[0,0].spines['left'].set_color('grey')
    ax[1,0].spines['left'].set_color('grey')
    
    plt.subplots_adjust(hspace= 0.1)


    #plt.title('Profile '+ pr, fontsize = 15)
    ax[1,0].set_xlabel("d (m)", fontsize = 9)
    #plt.ylabel("Mean Reflectance", fontsize = 15)
    ax[0,0].legend(loc = 'upper center')
    #plt.savefig(os.path.join(pathS, name))
plt.show()


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [19]:
def turning_points(array):
    
    idx_max, idx_min = [], []
    if (len(array) < 3):
        return idx_min, idx_max
    NEUTRAL, RISING, FALLING = range(3)
    
    def get_state(a,b):
        if a < b: return RISING
        if a > b: return FALLING
        return NEUTRAL
    ps = get_state(array[0], array[1])
    begin = 1
    for i in range(2, len(array)):
        s = get_state(array[i-1], array[i])
        if s!= NEUTRAL:
            if ps != NEUTRAL and ps != s:
                if s == FALLING:
                    idx_max.append((begin + i - 1) // 2)
                else:
                    idx_min.append((begin + i - 1) // 2)
            begin = i
            ps = s
    return idx_min, idx_max

In [20]:
idx_min, idx_max = turning_points(y_split)

In [21]:
indexes_turning = np.sort([0] + idx_min + idx_max + [len(y_split)])

In [22]:
len(distances)

NameError: name 'distances' is not defined

In [23]:
overlayed['length_sum'] = overlayed['length'].cumsum(axis = 0)
plotted_pix_idxs = np.arange(0,len(overlayed),1)

#overlayed.reset_index(inplace = True)
distances = []
distances.append([overlayed.iloc[f]['length_sum'] for f in plotted_pix_idxs])
arr_rgb = []

for n in [42, 22, 15]: #

    ff = arr[:,:,n]
    arr_rgb.append(ff)

arr_rgb = np.stack(arr_rgb)
arr_rgb = np.transpose(arr_rgb, [1,2,0])

gg = plt.imshow(arr_rgb)
  
plt.show()

array_clipped = gg.get_array()
colors1 = [(array_clipped[overlayed.iloc[f]['row'],overlayed.iloc[f]['col'],:]) for f in plotted_pix_idxs ]
#colors = [np.array(color/np.amax(arr_rgb)*255, np.int32) for color in colors1]

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [24]:
indexes_turning

array([  0,   9,  90, 102, 202, 214, 230, 246, 347, 375, 383, 407, 455])

In [25]:
x_seg

NameError: name 'x_seg' is not defined

In [34]:
arr_copy = arr_rgb.copy()

for idd, line in overlayed.iterrows():
        arr_copy[line['row'], line['col'], :] = [255, 255, 255]

my_labels = {'l1': 'Turning min', 'l2': 'Turning max', 'l3': r"$ min(\Delta \overline{R}/\Delta d$)"}

with plt.rc_context({'axes.edgecolor':'grey', 'xtick.color':'grey', 'ytick.color':'grey'}):

    fig, ax = plt.subplots(2,2, sharex = True, figsize = (14,6))
    #plt.scatter(idd_inter_new, inter, s=150, c='firebrick', alpha = 0.5, label = 'interface')
    ax[0,0].plot(x_new, y_new, marker = 'o', color='royalblue', label = 'mean reflectance') # royalblue
    #plt.plot(deriv_new/0.25, marker = '.', color = 'slategray', label = 'derived') # #  skyblue


    ax[0,0].plot(x_split,y_split, color = 'orange', label = 'Averaged R')
    ax[1,0].plot(x_split[:-1],deriv_new_split/0.25, marker = '.', color = 'steelblue', label = 'derived') # #  skyblue



    idds_inter = []
    inters_seg = []
    cols = []
    rows = []
    for a , b in zip(indexes_turning, indexes_turning[1:]):
        if b - a == 1:
            continue
        #ax.plot(np.arange(a,b,1), y_split[a:b]*1.5)
        ax[0,0].plot(x_split[a:b], y_split[a:b]*1.5)
        #deriv_seg = deriv_split[a:b]
        #idd_inter_new_seg = np.argmin(deriv_seg)
        #inter_seg = y_split[a + idd_inter_new_seg]
        #plt.scatter(a  + idd_inter_new_seg, inter_seg, s=20, c='olive', alpha = 0.5, label = my_labels['l3'])
        x_seg = x_split[a:b]
        y_seg = y_split[a:b]
        idd_inter_seg, inter_seg, x_new_seg, y_new_seg, deriv_new_seg = Interpolate_R_PR_Split(x_seg, y_seg)
        print(a,b, a+idd_inter_seg, idd_inter_seg, inter_seg) # x_new_seg[a + idd_inter_seg]
        idds_inter.append(a + idd_inter_seg)
        inters_seg.append(inter_seg)
        col, row = GetPixCoordinates(idd_inter_seg, inter_seg, x_new_seg, y_new_seg, deriv_new_seg , prx)
        cols.append(col)
        rows.append(row)

        ax[0,0].scatter(x_split[a  + idd_inter_seg], inter_seg, s=20, c='indigo', zorder = 3,alpha = 0.5, label = my_labels['l3'])
        
        my_labels['l3'] = '_nolegend_'
        
    differences = []
    # Calculate the differences between the value defining the foam and the segment derivative minimums
    for value in inters_seg:
        differences.append(value - dictio['I'])

    # Take the one with the higher difference

    #max_diff = np.sort(differences)[-1]

    for f in np.sort(differences):

        if f>0:
            print(f)
            break

    iddd = differences.index(f)
    idd_inter_new = idds_inter[iddd]
    inter = inters_seg[iddd]
    col, row = GetPixCoordinates(idd_inter_new, inter, x_new, y_new, deriv_new , prx)
            
    ax[0,0].scatter(x_split[idd_inter_new], inter, s = 155, c = 'teal')
    

        
     ## Anhadr de 0 a 1 color1[0]
    
    ax[0,0].axvspan(0, distances[0][0], alpha = 0.6, facecolor = colors1[0].data)
    ax[1,0].axvspan(0, distances[0][0], alpha = 0.6, facecolor = colors1[0].data)
    o = 1
    
    for diss, collor in zip(distances[0], colors1):
        #print(matplotlib.colors.to_hex(collor))
        
        if o == len(distances[0]):
            o = len(distances[0]) -1

        ax[0,0].axvspan(diss, distances[0][o], alpha = 0.6, facecolor = collor.data)
        ax[1,0].axvspan(diss, distances[0][o], alpha = 0.6, facecolor = collor.data)
        
        ax[0,0].axvline(diss, color= 'white', ls = 'dashed', linewidth = 0.2, zorder = 3)
        ax[1,0].axvline(diss, color= 'white', ls = 'dashed', linewidth = 0.2, zorder = 3)
        o += 1
        #ax[1].axvline(diss, color= sorted_names[(o*3)*5], ls = 'dashed', linewidth = 3, zorder = 0)


    for i in idx_min:
        ax[0,0].scatter(x_split[i], y_split[i]*1.5, s=20, c='firebrick', alpha = 0.5, label = my_labels['l1'], zorder = 3)
        my_labels['l1'] = "_nolegend_"
    for i in idx_max:
        ax[0,0].scatter(x_split[i], y_split[i]*1.5, s = 20, c='limegreen', alpha = 0.5, label = my_labels['l2'], zorder = 3 )
        my_labels['l2'] = "_nolegend_"



    plt.subplot(1,2,2)
    plt.imshow(arr_copy)
    plt.scatter(cols, rows, c= 'indigo', s = 10, alpha = 0.4, 
                edgecolor = 'red', label = r"$ min(\Delta \overline{R}/\Delta d$)")
    plt.scatter(col,row, s = 155, c = 'teal')
    plt.legend(loc = 'lower right', markerscale = 3)
    plt.axis('off')



    ax[0,0].spines['top'].set_visible(False)
    ax[1,0].spines['top'].set_visible(False)
    ax[0,0].spines['bottom'].set_color('grey')
    ax[1,0].spines['bottom'].set_color('grey')
    ax[0,0].spines['right'].set_visible(False)
    ax[1,0].spines['right'].set_visible(False)

    ax[0,0].spines['left'].set_color('grey')
    ax[1,0].spines['left'].set_color('grey')

    ax[0,0].set_ylabel(r"$\overline{R}$", fontsize = 20)
    ax[1,0].set_ylabel(r"$\Delta \overline{R}/\Delta d$", fontsize = 20)

    plt.subplots_adjust(hspace= 0.1)
    plt.title('Profile '+ pr, fontsize = 15)
    ax[1,0].set_xlabel("d (m)", fontsize = 15)
    plt.ylabel("Mean Reflectance", fontsize = 15)
    ax[0,0].legend(ncol = 2, loc='upper center')
#plt.savefig(os.path.join(pathS, name))
plt.show()


0 9 7 7 0.2843915069900848
9 90 80 71 0.27031292784119204
90 102 90 0 0.26683901602858917
102 202 189 87 0.14715690842519139
202 214 202 0 0.11724467971009662
214 230 221 7 0.12102929808245001
230 246 230 0 0.11718785526240048
246 347 284 38 0.09089287314818988
347 375 347 0 0.044508717581989335
375 383 379 4 0.06810371546132775
383 407 383 0 0.0664716513189484
407 455 453 46 0.023499296575158684
0.09557307535973772


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [27]:
differences

[]

In [28]:
inters_seg

[0.2843915069900848,
 0.27031292784119204,
 0.26683901602858917,
 0.14715690842519139,
 0.11724467971009662,
 0.12102929808245001,
 0.11718785526240048,
 0.09089287314818988,
 0.044508717581989335,
 0.06810371546132775,
 0.0664716513189484,
 0.023499296575158684]

In [29]:
len(differences) == len(inters_seg)

False

In [30]:
differences[iddd], inters_seg[iddd]

NameError: name 'iddd' is not defined

Load the information related to the profiles sds statistics

In [31]:
def Generate_Dict_Training_Dict(path, filename):
    info = pd.read_csv(os.path.join(path, filename))
    groups = info.groupby('Proxy')
    dictio = {}
    for idd, group in groups:
        dictio[idd] = group['Medians'].mean()
        
    return dictio

In [32]:
path0 = '/media/sf_VBox_Shared/PAOLA/PRISMA_FOAM/'
filename = 'Statistics_training_ProfilesSDS.csv'
dictio = Generate_Dict_Training_Dict(path0, filename)

In [842]:
if inter <= dictio['F'] :
    print('Hay que partir')

Hay que partir


In [48]:
if inter <= dictio['F']:
    print('This should be foam')
    # We have to discard this point and the data after
    
if inter >= dictio['I']:
    print('This should not be foma')


This should be foam


In [843]:
dictio['I']

{'F': 0.08865128496566196,
 'I': 0.17126594066885145,
 'IR': 0.1834132194343064,
 'MIF': 0.1262464629836433,
 'MIR': 0.166308811182435,
 'MRF': 0.08591849299742434,
 'R': 0.211492011626957}

In [784]:
x = np.arange(0, len(overlayed.index))
y = np.arange(0, 12)

In [805]:
fig, ax = plt.subplots(1,1, figsize = (12,6))
xx = x[::-1]
for i in xx:
    plt.plot([xx[i]] * len(y), y, color = colors1[i].data)
    
plt.show()